# TM-align Phylogenetic Tree Maker (working title)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crfield18/TMmatrix/blob/main/tmmatrix.ipynb)


Create a phylogenetic tree that compares the secondary structure of proteins, rather than nucleotide or amino acid sequence. Scoring uses the [_TM-align_](https://zhanggroup.org/TM-align/) algorithm by [_Zhang and Skolnick (2005)_](https://doi.org/10.1093%2Fnar%2Fgki524).

A score of **<0.17** indicates similarity indistinguishable from a random pair of structures, where as as score **≥0.50** indicates a pair with the same fold ([_Xu et al., 2010_](https://doi.org/10.1093/bioinformatics/btq066)).


Made possible with the use of: [Open-Source PyMol](https://github.com/schrodinger/pymol-open-source), [psico](https://github.com/speleo3/pymol-psico) and [Biopython](https://biopython.org).


### Usage

* Currently only working on _Google Chrome_ (I've had issues with the file upload step on _Safari_)

* Run each code block individually by clicking the play ▶ icon.

* Running the _Install Dependencies_ block will cause the kernal to restart, and an error stating _"Your session crashed for an unknown reason"_. This can be ignored. 

* **RECOMMENDED MAXIMUM # of models** is 125. This equates to 7750 TM-align calucations (eq. 1), which should take ~ 1 hour to complete. You are welcome to bypass this limit, just be aware that the number of TM-align calculations increases by _n - 1_ where _n_ is the number of PDB files.

> Equation 1. $$\frac{n!}{2(n-2)!}$$


In [ ]:
# @title Install Dependencies

!pip install -q condacolab

import condacolab
condacolab.install()

!mamba install conda-forge::pymol-open-source schrodinger::pymol-psico pandas conda-forge::biopython bioconda::tmalign

In [ ]:
# @title Upload PDB/mmCIF files

# Upload PDB/mmCIF files
from google.colab import files

uploaded = files.upload()

# Create subdirectory for PDB/mmCIF files and results
from pathlib import Path

cwd = Path.cwd()
for sub_dir in (cwd.joinpath('models'), cwd.joinpath('results')):
  sub_dir.mkdir(parents=True, exist_ok=True)

# Move PDB/mmCIF files to models subdirectory
import shutil

models_set = set(cwd.glob('*.pdb')).union(set(cwd.glob('*.cif')))

for file in models_set:
  source = file
  dest = cwd.joinpath(f'models/{file.name}')
  shutil.move(source, dest)

In [ ]:
# @title Estimate Run Time (optional)

from pathlib import Path

cwd = Path.cwd()

# Count the number of files with a .pd or .cif extension in the models directory
model_file_count = 0
for file in cwd.joinpath('models').glob('*.pdb'):
    model_file_count += 1
for file in cwd.joinpath('models').glob('*.cif'):
    model_file_count += 1

# Estimate time to complete the matrix
from math import factorial
def calculate_ncr(n:int=1):
    # n = number of models
    # r ≡ 2 since we're always dealing with pairs of models
    return (factorial(n)/(2*factorial(n-2)))

# Brief testing indicates the Colab server can complete ≈ 3 TM-align calculations per second
# This is dependent on the size of each model
estimate_in_s = calculate_ncr(n=model_file_count) / 3

# Output estimate
def time_string(time_in_s:int):
    secs = time_in_s

    hrs = time_in_s // 3600
    secs %= 3600
    mins = secs // 60
    secs %= 60

    return f'{hrs} hours {mins} mins {secs} secs'

print(f'Estimated time for completion for {model_file_count} models:\t{time_string(estimate_in_s)}')

In [ ]:
# @title Calculate TM-align Matrix

from itertools import combinations
from math import isnan
from pathlib import Path
import numpy as np
import pandas as pd
from pymol import cmd
import psico.fullinit # Needed to add TM-align to PyMOL

from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor, DistanceMatrix

class TMalignMatrix():
  def __init__(self) -> None:
    self.models = {}
    self.home_path = Path.cwd()
    self.models_path = self.home_path.joinpath('models')
    self.results_path = self.home_path.joinpath('results')
    self.tmmatrix = pd.DataFrame()

  def get_models(self):
    models_set = set(self.models_path.glob('*.pdb')).union(set(self.models_path.glob('*.cif')))
    for model in models_set:
      self.models[model.name] = model
    return self.models

  def load_models(self):
    if self.models == {}:
      self.get_models()
    for model, path in self.models.items():
      # Load each model into pymol. Objects are named the same as the model file without .pdb/.cif
      cmd.load(path, model[:-4])

    # Remove any waters
    cmd.remove('resn hoh')

  def get_sequence_length(self, object_name):
    # Count the number of CA atoms in given object
    sequence_length = cmd.count_atoms(f'{object_name} and name CA')
    return sequence_length

  def calculate_matrix(self):
    # Load all PDB files into pymol
    self.load_models()
    model_list = [model[:-4] for model, path in self.models.items() if model.endswith('.pdb') or model.endswith('.cif')]

    # Initialise an empty pandas dataframe using the list of loaded pdb files as column/row indeces
    if self.tmmatrix.empty:
      self.tmmatrix = pd.DataFrame(index=model_list, columns=model_list)

    # Calculate TM-align scores for all unique combinations of models
    for pair in combinations(model_list, 2):
      if isnan(self.tmmatrix.loc[pair[0], pair[1]]) or isnan(self.tmmatrix.loc[pair[1], pair[0]]):
        # Set the smaller object as the reference (target) object
        # TM-align scores are normalised based on sequence length
        object_1_length = self.get_sequence_length(pair[0])
        object_2_length = self.get_sequence_length(pair[1])

        if object_1_length >= object_2_length:
            target = pair[1]
            mobile = pair[0]
        elif object_1_length < object_2_length:
            target = pair[0]
            mobile = pair[1]

        tm = cmd.tmalign(mobile, target)
        self.tmmatrix.loc[pair[0], pair[1]] = tm
        self.tmmatrix.loc[pair[1], pair[0]] = tm

    # Fill the diagonal with TM-align scores of 1 (always the score for identical proteins)
    # ∴ We do not need to run any calculations
    for obj in model_list:
      self.tmmatrix.loc[obj,obj] = 1.0

    # Write the TM-align score matrix to a csv file
    self.tmmatrix.to_csv(self.results_path.joinpath('tm-align_score_matrix.csv'), index=True)

    return self.tmmatrix

  def make_tree_from_matrix(self):
    if self.tmmatrix.empty:
      distances_df = pd.read_csv(self.results_path.joinpath('tm-align_score_matrix.csv'), index_col=0)
    else:
      distances_df = self.tmmatrix

    # Invert TM-align scores
    # More similar pairs of models (i.e., higher scores) have shorter distances to each other
    distances_df = 1 - distances_df

    # Convert the scores matrix to a lower triangle matrix
    lower_tri_df = distances_df.where(np.tril(np.ones(distances_df.shape)).astype(bool))
    lower_tri_lists = [[value for value in row if not isnan(value)] for row in lower_tri_df.values.tolist()]

    # Generate phylogenetic tree
    tm_matrix = DistanceMatrix(names=distances_df.index.values.tolist(),matrix=lower_tri_lists)
    constructor = DistanceTreeConstructor()
    tree = constructor.upgma(tm_matrix)

    # Draw the tree for quick validation
    # Only label terminal nodes
    def no_internal_labels(node):
      if node.is_terminal():
        return node.name
      else:
        return None
    Phylo.draw(tree, label_func=no_internal_labels)
    # Phylo.draw_ascii(tree)

    # Write the tree to results/results.tree
    Phylo.write(tree, self.results_path.joinpath('results.tree'), 'newick')

    # Zip and Dowload results directory
    from google.colab import files
    import zipfile

    zip_path = self.results_path.joinpath('results.zip')

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
      for file in self.results_path.rglob('*'):
        if file.is_file() and file.name != 'results.zip':  # Add only files (not directories) to the ZIP archive
          zipf.write(file, arcname=file.relative_to(self.results_path))

    files.download(zip_path)

if __name__ == '__main__':
  colab_instance = TMalignMatrix()
  colab_instance.calculate_matrix()
  colab_instance.make_tree_from_matrix()